In [1]:
import json
import numpy as np
from math import degrees
import pandas as pd
import os

def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-6)
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return degrees(angle)

def count_tricep_pushdown_from_json(json_data, min_angle=45, max_angle=160):
    rep_count = 0
    state = "up"
    rep_frame_indices = []

    for frame_data in json_data["frames"]:
        frame_idx = frame_data["frame_index"]
        landmarks = frame_data.get("landmarks", [])

        if not landmarks or len(landmarks) < 33:  # Ensure full pose landmarks
            continue
        
        # Mediapipe landmark indexes for RIGHT_SHOULDER=12, RIGHT_ELBOW=14, RIGHT_WRIST=16
        # (Note: Indexing may differ by implementation, adjust if needed)
        right_shoulder = [landmarks[12]["x"], landmarks[12]["y"]]
        right_elbow = [landmarks[14]["x"], landmarks[14]["y"]]
        right_wrist = [landmarks[16]["x"], landmarks[16]["y"]]

        angle = calculate_angle(right_shoulder, right_elbow, right_wrist)

        if state == "up" and angle > max_angle:
            rep_count += 1
            rep_frame_indices.append(frame_idx)
            state = "down"
        elif state == "down" and angle < min_angle:
            state = "up"

    return rep_count, rep_frame_indices

def process_json_files_in_folder(folder_path, output_excel_path, min_angle=45, max_angle=160):
    json_files = [f for f in os.listdir(folder_path) if f.endswith(".json")]
    records = []

    for json_file in json_files:
        json_path = os.path.join(folder_path, json_file)
        print(f"Processing: {json_file}")
        try:
            with open(json_path, 'r') as f:
                data = json.load(f)

            rep_count, rep_frames = count_tricep_pushdown_from_json(data, min_angle, max_angle)
            record = {
                "File_Name": json_file,
                "Video_Name": data.get("video_name", "Unknown"),
                "Rep_Count": rep_count,
                "Rep_Frames": ", ".join(map(str, rep_frames)) if rep_frames else "None"
            }
            records.append(record)
        except Exception as e:
            print(f"Error processing {json_file}: {e}")
            records.append({
                "File_Name": json_file,
                "Video_Name": "Error",
                "Rep_Count": "Error",
                "Rep_Frames": str(e)
            })

    df = pd.DataFrame(records)
    df.to_excel(output_excel_path, index=False)
    print(f"\n✅ Results saved to: {output_excel_path}")

# === Usage ===
if __name__ == "__main__":
    json_folder = r"C:\Users\bdsid\OneDrive\Desktop\Dune Tech\Rep Counting\Data\Test_Bhaskar_Sir\tricep_pushdown\Json"
    output_excel =r"C:\Users\bdsid\OneDrive\Desktop\Dune Tech\Rep Counting\Data\Test_Bhaskar_Sir\tricep_pushdown\Json\rep_counts_tricep_pushdowns_from_json.xlsx"
    process_json_files_in_folder(json_folder, output_excel, min_angle=45, max_angle=160)


Processing: tricep_Pushdown_segment_10_start_34m06s400ms_pose.json
Error processing tricep_Pushdown_segment_10_start_34m06s400ms_pose.json: list indices must be integers or slices, not str

✅ Results saved to: C:\Users\bdsid\OneDrive\Desktop\Dune Tech\Rep Counting\Data\Test_Bhaskar_Sir\tricep_pushdown\Json\rep_counts_tricep_pushdowns_from_json.xlsx


In [1]:
import os
import json
import numpy as np
import pandas as pd
from math import degrees

def calculate_angle(a, b, c):
    """Calculate angle at point 'b' given 3 points a-b-c."""
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba = a - b
    bc = c - b
    cosine = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-6)
    angle = np.arccos(np.clip(cosine, -1.0, 1.0))
    return degrees(angle)

def extract_keypoints(landmarks):
    """Map landmark names to (x, y) positions."""
    return {lm["name"]: (lm["x"], lm["y"]) for lm in landmarks if lm["conf"] > 0.5}

def count_tricep_pushdowns_from_json(json_folder, min_angle=45, max_angle=160):
    json_files = sorted([f for f in os.listdir(json_folder) if f.endswith('.json')])
    rep_count = 0
    state = "up"
    rep_frames = []

    for fname in json_files:
        fpath = os.path.join(json_folder, fname)
        with open(fpath, 'r') as f:
            data = json.load(f)

        keypoints = extract_keypoints(data.get("landmarks", []))

        if all(k in keypoints for k in ["right_shoulder", "right_elbow", "right_wrist"]):
            angle = calculate_angle(
                keypoints["right_shoulder"],
                keypoints["right_elbow"],
                keypoints["right_wrist"]
            )

            if state == "up" and angle > max_angle:
                rep_count += 1
                rep_frames.append(data.get("frame", fname))
                state = "down"
            elif state == "down" and angle < min_angle:
                state = "up"

    return rep_count, rep_frames

def process_json_folders(folder_path, output_csv_path, min_angle=45, max_angle=160):
    folders = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]
    records = []

    for folder in folders:
        full_path = os.path.join(folder_path, folder)
        print(f"Processing: {folder}")
        try:
            reps, frames = count_tricep_pushdowns_from_json(full_path, min_angle, max_angle)
            records.append({
                "Session": folder,
                "Rep_Count": reps,
                "Rep_Frames": ", ".join(map(str, frames)) if frames else "None"
            })
        except Exception as e:
            print(f"❌ Error in {folder}: {e}")
            records.append({
                "Session": folder,
                "Rep_Count": "Error",
                "Rep_Frames": str(e)
            })

    pd.DataFrame(records).to_csv(output_csv_path, index=False)
    print(f"\n✅ Rep count saved to: {output_csv_path}")

# === USAGE ===
if __name__ == "__main__":
    json_root_folder = r"C:\Users\bdsid\OneDrive\Desktop\Dune Tech\Rep Counting\Data\Test_Bhaskar_Sir\tricep_pushdown\Json"
    output_csv = r"C:\Users\bdsid\OneDrive\Desktop\Dune Tech\Rep Counting\Data\Test_Bhaskar_Sir\tricep_pushdown\Json\output_tricep_pushdown_reps.csv"
    process_json_folders(json_root_folder, output_csv, min_angle=45, max_angle=160)



✅ Rep count saved to: C:\Users\bdsid\OneDrive\Desktop\Dune Tech\Rep Counting\Data\Test_Bhaskar_Sir\tricep_pushdown\Json\output_tricep_pushdown_reps.csv
